In [11]:
import rdflib
import os, sys

text = sys.argv[1]

def load_graph():
    g = rdflib.Graph()
    g.parse("onto.nt", format="nt")
    return g

def check_in_ontology(g, a, b):
    q = "select * where {<http://o.org/"+a+"> ?p <http://o.org/"+b+">}"
    x1 = g.query(q)
    for x in x1:
        print()
        print(a,x,b)
        get_type(g, a)
        get_type(g, b)
    
    return x1


def get_type(g, a):
    q = "select ?t where {<http://o.org/"+a+"> <http://o.org/type> ?t}"
    x1 = g.query(q)
    for x in x1:
        print(a,x)


def normalize_word(w):
    # TODO: stemming
    return w.replace(",","").replace(".","").strip()
    
def check_text(g, text):
    text = text.split()
    for t1 in text:
        t1 = normalize_word(t1)
        for t2 in text:
            t2 = normalize_word(t2)
            if t1 != t2:
                check_in_ontology(g, t1, t2)
                
def geographic_difference(g, s, t):
    # Currently - equal weight for every connection
    s = s.lower().replace(" ", "_").split(",")[0].strip()
    t = t.lower().replace(" ", "_").split(",")[0].strip()
    if s == t:
        return 0
    return 3 - min(len(check_in_ontology(g, s, t)), 3)

def month_to_num(m):
    if m == "jan":
        return 1
    if m == "feb":
        return 2
    if m == "mar":
        return 3
    if m == "apr":
        return 4
    if m == "may":
        return 5
    if m == "jun":
        return 6
    if m == "jul":
        return 7
    if m == "aug":
        return 8
    if m == "sep":
        return 9
    if m == "oct":
        return 10
    if m == "nov":
        return 11
    if m == "dec":
        return 12
    

def date_difference(s, t):
    s_splitted = s.lower().split()
    t_splitted = t.lower().split()
    s_m = month_to_num(s_splitted[0][0:3])
    s_d = s_splitted[1]
    t_m = month_to_num(t_splitted[0][0:3])
    t_d = t_splitted[1]
    
    #print(s_m, s_d, t_m, t_d)

    return abs((s_m*30+int(s_d)) - (t_m*30+int(t_d)))

g = load_graph()
print(geographic_difference(g, "paris", "london"))



paris (rdflib.term.URIRef('http://o.org/same_continent'),) london

paris (rdflib.term.URIRef('http://o.org/same_country'),) london

paris (rdflib.term.URIRef('http://o.org/near_by'),) london
0


In [12]:
import json

def load_json(frame_path):
    chat_file = open(frame_path, encoding='utf-8')
    chat = json.load(chat_file)
    return chat

final_vector = []
def prepare_frames_vector(chat):
    dst_city = or_city = str_date = end_date = ""
    or_city_diff = dst_city_diff = str_date_diff = end_date_diff = 0
    for t in chat["turns"]:
        if "text" in t:
            prev_or_city = or_city
            prev_dst_city = dst_city
            prev_str_date = str_date
            prev_end_date = end_date
            
            for f in t["labels"]["frames"]:
                #print(f["info"])
                info = f["info"]
                if "or_city" in info:
                    or_city = info["or_city"][0]["val"]
                if "dst_city" in info:
                    dst_city = info["dst_city"][0]["val"]
                if "str_date" in info:
                    str_date = info["str_date"][0]["val"]
                if "end_date" in info:
                    end_date = info["end_date"][0]["val"]

            if len(prev_or_city) > 1:
                or_city_diff = geographic_difference(g, prev_or_city, or_city)
            if len(prev_dst_city) > 1:
                dst_city_diff = geographic_difference(g, prev_dst_city, dst_city)
                
            if len(prev_str_date) > 1:
                str_date_diff = date_difference(str_date, prev_str_date)
            if len(prev_end_date) > 1:
                end_date_diff = date_difference(end_date, prev_end_date)

            print()
            print(t["author"])
            print(t["text"])                
            print(or_city_diff, dst_city_diff, str_date_diff, end_date_diff)
            
            final_vector.append([t["author"], t["text"], or_city_diff+dst_city_diff, str_date_diff+end_date_diff])
    
    return final_vector
                    
#chat = load_json("../Frames-dataset/chats/00d28996-ed48-4f09-800e-677bae20547f.json")
#chat = load_json("../Frames-dataset/chats/0ac64405-3e57-4f5a-96d2-9c564e78d6e5.json")
chat = load_json("../Frames-dataset/chats/0ee176c9-186d-4f6f-9db3-06edd35b1770.json")
print()
print(prepare_frames_vector(chat))
    



user
hay i am doing some business globetrotting just me and a colleague. we need to get to Fukuoka some time between august 27 and september 6
0 0 0 0

wizard
Would you be able to leave from Osaka? I could offer you 4 nights at the Historic Mansion Hotel for 483.14USD.
0 0 0 0

user
I can’t do Osaka. anything available from London tho?
0 0 0 0

wizard
I can fit in a stay at the Historic Mansion Hotel from the 30th of August to the 2nd of September. Leaving from London will bring your price up to 3250.48USD. Is this ok?
3 0 3 4

user
that would work! but before we book it I also want to look at a trip to Mannheim - how much would that be, and could you fit it between aug 27 and sept 6
0 3 3 4

wizard
Yes I can! You would be staying at the Regal Resort from the 27th to the 31st of August. The cost of this package is 1971.39USD.
0 0 0 5

user
ok! is that with my one colleauge included ?
0 0 0 0

wizard
The same trip for two would be 3942.78USD. WOuld you like me to book this trip?
0 0 0